In [ ]:
#looks good, only made minor edits to optimize functionality

import os
from openai import OpenAI

#os.environ["PERPLEXITY_API_KEY"] = '' # put API key here

client = OpenAI(
    api_key=os.environ["PERPLEXITY_API_KEY"],
    base_url="https://api.perplexity.ai"
)

def perplexity(prompt, system_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]

    try:
        response = client.chat.completions.create(
            model="sonar",  
            messages=messages,
            temperature=0 # for higher consistency 
        )
        return response.choices[0].message.content.strip(), []  
    except Exception as e:
        print("Error:", e)
        return "", []


In [ ]:
### FOR THE URLs ###

import pandas as pd
import re
from tqdm import tqdm
import time

df = pd.read_csv("10k_movies.csv")
batch_size = 100

system_prompt = (
    "You are a URL finder. You provide movie urls from letterboxd.com based on a movie's title and release year. "
    "Only return a valid Letterboxd URL that leads directly to the movie's page. "
    "Do not return any other URLs (such as person profiles, lists, or search results). The URL likely contains 'letterboxd.com/film/'"
    "Be strict about the release year. Do not return movies from a different year. "
    "If you're not sure, return NO URL FOUND"
)

urls = []

# === processing in batches ===
for start in range(0, len(df), batch_size):
    end = min(start + batch_size, len(df))
    batch = df.iloc[start:end]
    print(f"\nProcessing batch {start} to {end}")

    for index, row in tqdm(batch.iterrows(), total=len(batch)):
        title = row['title']
        release_date = row['release_date']

        prompt = f"Give me the Letterboxd URL for the movie: {title} ({release_date})"

        # === retry logic for perplexity call === 
        for attempt in range(3):
            try:
                result, _ = perplexity(prompt, system_prompt) 
                break
            except Exception as e:
                print(f"Error: {e} (attempt {attempt + 1})")
                time.sleep(3)
            else:
                result = ""

        time.sleep(1.5)
        print(result)

        # Use regex to find letterboxd URLs directly
        if result:
            letterboxd_urls = re.findall(r'https://letterboxd\.com/film/[^\s\[\]\)]+', result)
            if letterboxd_urls:
                url = letterboxd_urls[0].rstrip('/')  # Remove trailing slash if present
                urls.append(url)
                
                # Write the URL immediately to the file 
                with open("letterboxd_urls.txt", "a") as f:
                    f.write(url + "\n")

print("Saved clean URL list to 'letterboxd_urls.txt'")


The Letterboxd URL for the movie **Time for Revenge (1981-07-30)** is:

https://letterboxd.com/film/time-for-revenge/ 

This page corresponds exactly to the 1981 release directed by Adolfo Aristarain[3][5].
NO URL FOUND
NO URL FOUND
The Letterboxd URL for the movie "Japanese Borscht" (2019-07-30) is:

https://letterboxd.com/film/japanese-borscht/
The Letterboxd URL for the movie "Japanese Borscht" (2019-07-30) is:

https://letterboxd.com/film/japanese-borscht/
The Letterboxd URL for the movie "Taxi Ballad" released in 2012 is:

https://letterboxd.com/film/taxi-ballad/ 

This matches the 2012 release year you specified[3][4][5].
The Letterboxd URL for the movie "Taxi Ballad" released in 2012 is:

https://letterboxd.com/film/taxi-ballad/ 

This matches the 2012 release year you specified[3][4][5].
The Letterboxd URL for the movie "My Old Ass" (2024) is:

https://letterboxd.com/film/my-old-ass/ 

This matches the 2024 release year and leads directly to the film's page on Letterboxd[1].
Sa